In [1]:
import spacy
nlp = spacy.load(r'en_core_web_sm')
ent= nlp(text)
for word in ent.ents:
    print(word.text,word.label_)

In [2]:
nlp = spacy.load(r'en_core_web_sm')

In [3]:
text= 'A Special Police Officer (SPO), identified as Sameer Ahmad was killed by suspected militants at Hajan in Pulwama District on January 1 evening, reports Daily Excelsior. Sources said that suspected militants fired at Sameer Ahmad. “The injured was immediately taken to a hospital where he was declared as brought dead,” sources said. They said the militants managed to escape under the cover of darkness. “Security forces were rushed and a search operation was launched to nab the attackers,” they added'

In [4]:
ent= nlp(text)

for word in ent.ents:
    print(word.text,word.label_)

In [6]:
text2='Jammu and Kashmir  Desapani  Armed Assault  Police  Security Patrol  Separatists  Firearms  Firearm'

In [7]:
ent2= nlp(text2)

In [8]:
for word in ent2.ents:
    print(word.text,word.label_)

Jammu GPE
Kashmir LOC


In [9]:
from __future__ import unicode_literals, print_function

import plac #  wrapper over argparse
import random
from pathlib import Path
import spacy
from tqdm import tqdm # loading bar

In [69]:
LABEL = 'ATK'

In [70]:
TRAIN_DATA = [
    ("Bullet are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ATK')]
    }),

    ("Do they bite?", {
        'entities': []
    }),

    ("Bullet are too tall and they pretend to care about your feelings", {
        'entities': [(0, 6, 'ATK')]
    }),

    ("Bullet pretend to care about your feelings", {
        'entities': [(0, 6, 'ATK')]
    }),

    ("they pretend to care about your feelings, those Bullet", {
        'entities': [(48, 54, 'ATK')]
    }),

    ("Bullet?", {
        'entities': [(0, 6, 'ATK')]
    })
]

In [71]:
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    new_model_name=("New model name for model meta.", "option", "nm", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int))


def main(model=None, new_model_name='animal', output_dir=None, n_iter=20):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")
    # Add entity recognizer to model if it's not in the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
    # otherwise, get it, so we can add labels to it
    else:
        ner = nlp.get_pipe('ner')

    ner.add_label(LABEL)   # add new entity label to entity recognizer
    if model is None:
        optimizer = nlp.begin_training()
    else:
        # Note that 'begin_training' initializes the models, so it'll zero out
        # existing entity types.
        optimizer = nlp.entity.create_optimizer()

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in tqdm(TRAIN_DATA):
                nlp.update([text], [annotations], sgd=optimizer, drop=0.35,
                           losses=losses)
            print(losses)

    # test the trained model
    test_text = 'Do you like Bullet?'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        doc2 = nlp2(test_text)
        for ent in doc2.ents:
            print(ent.label_, ent.text)



In [72]:
main()

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Created blank 'en' model


 33%|████████████████████████████                                                        | 2/6 [00:00<00:00, 18.36it/s]

{'ner': 33.91313946247101}


 33%|████████████████████████████                                                        | 2/6 [00:00<00:00, 15.63it/s]

...

{'ner': 1.1759358845990357e-07}


 33%|████████████████████████████                                                        | 2/6 [00:00<00:00, 14.93it/s]

{'ner': 2.2793354886726512e-06}


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 15.59it/s]

{'ner': 7.829872301330754e-08}
Entities in 'Do you like Bullet?'
ATK Bullet


In [74]:
    test_text = 'hit Bullet?'
    doc = nlp(test_text)
    print("Entities in '%s'" % test_text)
    for ent in doc.ents:
        print(ent.label_, ent.text)

Entities in 'hit Bullet?'
ORG Bullet


In [18]:
import requests
from bs4 import BeautifulSoup
import spacy
nlp = spacy.load(r'en_core_web_sm')

USER_AGENT = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

################################# Google search Results ##################################################

def fetch_results(search_term, number_results, language_code):
    assert isinstance(search_term, str), 'Search term must be a string'
    assert isinstance(number_results, int), 'Number of results must be an integer'
    escaped_search_term = search_term.replace(' ', '+')

    google_url = 'https://www.google.com/search?q={}&num={}&hl={}'.format(escaped_search_term, number_results, language_code)
    response = requests.get(google_url, headers=USER_AGENT)
    response.raise_for_status()

    return search_term, response.text

if __name__ == '__main__':
    data='pulwama attack'
    keyword, html = fetch_results(data, 10, 'en')
  #  print(html)

########################################## Parse Text of each page ########################################

def parse_results(html, keyword):
    soup = BeautifulSoup(html, 'html.parser')

    found_results = []
    rank = 1
    result_block = soup.find_all('div', attrs={'class': 'g'})
    cardDetails = soup.find('div', attrs={'class': 'i4J0ge'})
#     print(cardDetails.get_text())

#     for a in cardDetails:
#         [x.extract() for x in a.findAll('h3')]
#         a.replaceWith(" %s " % a.string)
#         print(a.get_text())
    for result in result_block:

        link = result.find('a', href=True)
        title = result.find('h3', attrs={'class': 'LC20lb'})
        best_result = result.find('span',attrs={'class':'ILfuVd'})
        description = result.find('span', attrs={'class': 'st'})
        if link and title:
            link = link['href']
            title = title.get_text()
            if description:
                description = description.get_text()
            if link != '#':
                found_results.append({'keyword': keyword, 'rank': rank, 'title': title, 'description': description,'link':link})
                rank += 1
   
    return found_results
result=parse_results(html,'attack')

################################################# link Access  #############################################
wholetxt =""
def pageparser(link):
    global wholetxt
    if 'satp' in link:
        return
    else:
        print(link)
        response = requests.get(link)
        soup = BeautifulSoup(response.text,"html.parser")
        para=soup.find_all('p')
        for p in para:
            wholetxt += p.get_text() + " "
#         tokens =  nltk.word_tokenize(p.get_text())
#         text = nltk.Text(tokens)
# #         print(text)
#         text.concordance('messi')
    



for results in result:
    print(results['link'])
    pageparser(results['link'])
    

    
import re
# Removing Square Brackets and Extra Spaces
wholetxt = re.sub(r'\[[0-9]*\]', ' ', wholetxt)  
wholetxt = re.sub(r'\s+', ' ', wholetxt) 

# Removing special characters and digits
formatted_article_text = re.sub('[^a-zA-Z]', ' ', wholetxt )  
formatted_article_text = re.sub(r'\s+', ' ', wholetxt)
    
    
    
    
##############################################Summary###################################
import nltk
# nltk.download()
sentence_list = nltk.sent_tokenize(formatted_article_text)
stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}  
for word in nltk.word_tokenize(wholetxt):  
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():  
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    
import heapq  
sentence_scores = {}  
for sent in sentence_list:  
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)  
print(summary)

######################################## NER ##############################################
text=summary
ent= nlp(text)
for word in ent.ents:
    print(word.text,word.label_)
    


https://en.wikipedia.org/wiki/2019_Pulwama_attack
https://en.wikipedia.org/wiki/2019_Pulwama_attack
https://timesofindia.indiatimes.com/topic/pulwama-attack
https://timesofindia.indiatimes.com/topic/pulwama-attack
https://economictimes.indiatimes.com/topic/pulwama-Attack
https://economictimes.indiatimes.com/topic/pulwama-Attack
https://www.business-standard.com/topic/pulwama-attack
https://www.business-standard.com/topic/pulwama-attack
https://www.bbc.co.uk/news/topics/cgmkz7g3xn0t/pulwama-attack
https://www.bbc.co.uk/news/topics/cgmkz7g3xn0t/pulwama-attack
https://www.bbc.com/news/world-asia-india-47302467
https://www.bbc.com/news/world-asia-india-47302467
https://www.livemint.com/topic/pulwama-terror-attack
https://www.livemint.com/topic/pulwama-terror-attack
https://m.economictimes.com/news/defence/pulwama-attack-masterminds-aide-conspired-terror-attacks-across-india-nia-tells-court/articleshow/71153236.cms
https://m.economictimes.com/news/defence/pulwama-attack-masterminds-aide-con

In [22]:
import nltk
# nltk.download()
sentence_list = nltk.sent_tokenize(wholetxt)
stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}  
for word in nltk.word_tokenize(wholetxt):  
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():  
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    
import heapq  
sentence_scores = {}  
for sent in sentence_list:  
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)  
print(summary)

text=summary
ent= nlp(text)
for word in ent.ents:
    print(word.text,word.label_)

Support also poured in from Australia, European Union, Nepal, Bhutan, Sri Lanka, Afghanistan, Maldives, Bangladesh and a host of other countries. According to reports, India held a briefing for envoys of 25 countries including P5 nations, including US, China, Russia, UK and France. Following the attack on the Indian territory, the producers of the Indian Hindi films, including Notebook, Kabir Singh and Satellite Shankar, decided not to release the films in Pakistan. One of those colleges, Alpine College, suspended its dean, who is a Kashmiri, after some groups called for him to be fired. Vladimir Putin thanked Trump "for the transfer, through special services, of information allowing the prevention of terrorist acts in Russia," the president's office said in a statement. Initial investigations suggested the car was carrying more than 300 kilograms (660 lb) of explosives, including 80 kilograms (180 lb) of RDX, a high explosive, and ammonium nitrate. Adil Ahmad Dar was identified hours 

# pymongo creating database

In [19]:
from pymongo import MongoClient 
client = MongoClient('mongodb://localhost:27017/') 
mydatabase = client['databse']
mycollection = mydatabase['terrorist']
x = mycollection.insert_one(dic)

In [20]:
dic={}
for word in ent.ents:
    if word.label_ in dic:
        dic[word.label_]+=", "+(word.text)
    else:
        dic[word.label_]=word.text

In [21]:
dic

{'GPE': 'Australia, Nepal, Bhutan, Sri Lanka, Afghanistan, Maldives, Bangladesh, India, US, China, Russia, UK, France, Notebook, Pakistan, Kashmiri, Russia, Pakistan',
 'ORG': 'European Union, Alpine College, RDX',
 'CARDINAL': '25, One, 660',
 'NORP': 'Indian, Indian',
 'PERSON': 'Hindi, Kabir Singh, Satellite Shankar, Vladimir Putin, Adil Ahmad Dar, Jaish-e-Mohammad',
 'WORK_OF_ART': 'Trump',
 'QUANTITY': 'more than 300 kilograms, 80 kilograms, 180 lb'}

In [23]:
from nltk.tokenize import  word_tokenize
gpe_list=word_tokenize(dic['GPE'])
print(gpe_list[0])

Australia


In [26]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')
location = geolocator.geocode(gpe_list[0])
dic['lat,long']=location.latitude,location.longitude

GeocoderTimedOut: Service timed out

In [28]:
dic

{'GPE': 'Australia, Nepal, Bhutan, Sri Lanka, Afghanistan, Maldives, Bangladesh, India, US, China, Russia, UK, France, Notebook, Pakistan, Kashmiri, Russia, Pakistan',
 'ORG': 'European Union, Alpine College, RDX',
 'CARDINAL': '25, One, 660',
 'NORP': 'Indian, Indian',
 'PERSON': 'Hindi, Kabir Singh, Satellite Shankar, Vladimir Putin, Adil Ahmad Dar, Jaish-e-Mohammad',
 'WORK_OF_ART': 'Trump',
 'QUANTITY': 'more than 300 kilograms, 80 kilograms, 180 lb'}

In [32]:
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27017/')
mydatabase = client['databse']
mycollection = mydatabase['terrorist']
x = mycollection.insert_one(dic)

DuplicateKeyError: E11000 duplicate key error collection: databse.terrorist index: _id_ dup key: { _id: ObjectId('5e01bbde6f7bcb5ab95b9474') }